<a href="https://colab.research.google.com/github/viicctor91/me.github.io/blob/main/Projeto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJETO FINAL

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from tqdm import tqdm
import dask.dataframe as dd
from dask.distributed import Client
from sklearn.model_selection import train_test_split
#from pandas_profiling import ProfileReport
%matplotlib inline

pd.set_option('display.max_columns', None)

In [ ]:
# !pip install pandas_profiling

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows')

In [2]:
#lendo arquivos
df_sales = pd.read_csv("/content/drive/MyDrive/ARQUIVOS TFM/item_sales.csv")

In [3]:
df_prices = pd.read_csv("/content/drive/MyDrive/ARQUIVOS TFM/item_prices.csv")

In [4]:
df_calendar = pd.read_csv("/content/drive/MyDrive/ARQUIVOS TFM/daily_calendar_with_events.csv")

-------

### Dicionário de dados

File 1 – daily_calendar_with_events.csv

| Name | Description |
| --- | --- |
| date | date in y-m-d format |
| weekday |	day of the week |
| weekday_int |	numeric day of the week (Saturday day 1, Friday day 7) |
| d	| day identifier |
| event | if the date includes an event, the name of this event (only a few are included) |

File 2 – item_prices.csv

| Name | Description |
| --- | --- |
| item | product id |
| category | product category |
| store_code | alphanumeric code of the store |
| yearweek | date period for the price (year-week format) |
| sell_price |	price for the product “item” for the period in “yearweek”. * |

* Prices are provided per week (average across 7 days). If not available, there were no sales for the product during that week ludes an event, the name of this event (only a few are included).

File 3 – item_sales.csv

| Name | Description |
| --- | --- |
| id | sales series id (combination of item + store_code) |
| item | product id |
| category | product category |
| department | department id (different identifier for different stores) |
| store | store name |
| store_code | store id |
| region | region |
| d_1,d_2,d_... | number of units sold per day |

------

# Item Prices

## Pré-processamento e exploração de dados

In [ ]:
#Monstra Colonas do datafrane item_prices
cols_df_price = pd.DataFrame(df_prices.columns, columns=['Column Names'])

display(cols_df_price)

In [ ]:
# Primeiras cinco linhas do df
df_prices.head().T

In [ ]:
# Cinco ultimas linhas
df_prices.tail().T

In [17]:
# Dimensão do df
df_prices.shape

(6721786, 5)

In [18]:
df_prices.dtypes

item           object
category       object
store_code     object
yearweek        int64
sell_price    float64
dtype: object

In [ ]:
df_prices.describe()

In [ ]:
df_prices.info(verbose=True)

In [ ]:
#verificando valores em falta
soma_missings_por_coluna = df_prices.isnull().sum()
soma_missings_por_coluna

### Tratamento da coluna Yearweek

In [5]:
# Convertendo a coluna 'yearweek' para o tipo string
df_prices['yearweek'] = df_prices['yearweek'].astype(str)

# Remover linhas com valores vazios na coluna 'yearweek'
df_prices.dropna(subset=['yearweek'], inplace=True)

# Elimine as linhas que contenham a string 'nan' na coluna 'yearweek'
df_prices = df_prices[~df_prices['yearweek'].isin(['nan'])]

# Mantenha apenas os 6 primeiros caracteres na coluna 'yearweek'
df_prices['yearweek'] = df_prices['yearweek'].str.slice(0, 6)

# Ordenando o dataframe pela coluna yearweek
df_prices.sort_values('yearweek', inplace=True)

# Exiba o DataFrame reordenado
print(df_prices)

                        item       category store_code yearweek  sell_price
5539160    SUPERMARKET_3_702    SUPERMARKET      PHI_1   201105      3.9360
6613947  HOME_&_GARDEN_2_445  HOME_&_GARDEN      PHI_3   201105      8.1000
261069   HOME_&_GARDEN_2_041  HOME_&_GARDEN      NYC_1   201105      6.5875
1516539  HOME_&_GARDEN_1_117  HOME_&_GARDEN      NYC_3   201105     11.2125
563249     SUPERMARKET_3_188    SUPERMARKET      NYC_1   201105      2.3760
...                      ...            ...        ...      ...         ...
2386874  HOME_&_GARDEN_2_338  HOME_&_GARDEN      NYC_4   201617      6.1500
2387152  HOME_&_GARDEN_2_339  HOME_&_GARDEN      NYC_4   201617      8.0750
6156810    SUPERMARKET_3_382    SUPERMARKET      PHI_2   201617      2.1360
2386079  HOME_&_GARDEN_2_334  HOME_&_GARDEN      NYC_4   201617      3.7250
6965697    SUPERMARKET_3_827    SUPERMARKET      PHI_3   201617      1.2000

[6721786 rows x 5 columns]


In [ ]:
# Verificando se ainda existe valores nulos no df
df_prices.isnull().sum()/len(df_prices)*100

In [ ]:
#verificando valores duplicador no df
df_prices.duplicated().sum()/len(df_prices)*100

### Função para observar cada coluna do df

In [6]:
def Price_func(data, column, count=True):
    unique_values = data[column].unique()
    null_count = data[column].isnull().sum()
    value_counts = data[column].value_counts()

    print(f'Quantidade de valores únicos: {len(unique_values)}')
    print(f'\nQuais são os valores únicos: {unique_values}')
    print(f'\nQuantidade de valores nulos: {null_count}')
    print(f'\nQuantidade por opção: \n{value_counts}')

#### Coluna Item

In [ ]:
Price_func(df_prices, 'item')

Quantidade de valores únicos: 3049

Quais são os valores únicos: ['SUPERMARKET_3_702' 'HOME_&_GARDEN_2_445' 'HOME_&_GARDEN_2_041' ...
 'SUPERMARKET_3_647' 'SUPERMARKET_3_038' 'HOME_&_GARDEN_1_159']

Quantidade de valores nulos: 0

Quantidade por opção: 
SUPERMARKET_3_702      2790
HOME_&_GARDEN_1_234    2790
ACCESORIES_1_194       2790
SUPERMARKET_1_168      2790
SUPERMARKET_3_590      2790
                       ... 
HOME_&_GARDEN_1_308     572
HOME_&_GARDEN_1_159     553
HOME_&_GARDEN_1_242     530
SUPERMARKET_3_296       522
SUPERMARKET_2_379       463
Name: item, Length: 3049, dtype: int64


#### Coluna Category

In [ ]:
Price_func(df_prices, 'category')

#### Coluna Store_code

In [7]:
Price_func(df_prices, 'store_code')

Quantidade de valores únicos: 10

Quais são os valores únicos: ['PHI_1' 'PHI_3' 'NYC_1' 'NYC_3' 'BOS_3' 'BOS_1' 'PHI_2' 'BOS_2' 'NYC_4'
 'NYC_2']

Quantidade de valores nulos: 0

Quantidade por opção: 
BOS_2    689568
BOS_1    688135
NYC_1    686681
PHI_3    684355
NYC_3    682193
BOS_3    679290
NYC_4    666983
PHI_2    666154
PHI_1    653817
NYC_2    624610
Name: store_code, dtype: int64


#### Coluna Yearweek

In [ ]:
Price_func(df_prices, 'yearweek')

#### Coluna Sell_price

In [ ]:
Price_func(df_prices, 'sell_price')

Quantidade de valores únicos: 1860

Quais são os valores únicos: [ 3.936   8.1     6.5875 ... 18.05   34.5268  8.5519]

Quantidade de valores nulos: 0

Quantidade por opção: 
2.3760     213714
3.5760     208588
3.0000     182471
1.2000     145538
4.7760     137539
            ...  
0.7500          1
65.7750         1
3.5910          1
0.4522          1
8.5519          1
Name: sell_price, Length: 1860, dtype: int64


### Certificar-se de que todos os valores na coluna sell_price sejam do tipo float.
###  Arredondar a coluna sell_price para duas casas decimais.

In [8]:
# 1. Certificar-se de que todos os valores na coluna sell_price sejam do tipo float
df_prices['sell_price'] = df_prices['sell_price'].astype(float)

# 2. Arredondar a coluna sell_price para duas casas decimais
df_prices['sell_price'] = df_prices['sell_price'].round(2)


In [8]:
price_stats = df_prices.groupby("category")["sell_price"].agg(["mean", "median", "std"])
print(price_stats)

                   mean  median       std
category                                 
ACCESORIES     7.084556    5.28  6.407341
HOME_&_GARDEN  6.831656    6.18  4.215563
SUPERMARKET    3.899546    3.22  2.555709


In [10]:
df_prices.head()

,item,category,store_code,yearweek,sell_price
5539160,SUPERMARKET_3_702,SUPERMARKET,PHI_1,201105,3.94
6613947,HOME_&_GARDEN_2_445,HOME_&_GARDEN,PHI_3,201105,8.10
261069,HOME_&_GARDEN_2_041,HOME_&_GARDEN,NYC_1,201105,6.59
1516539,HOME_&_GARDEN_1_117,HOME_&_GARDEN,NYC_3,201105,11.21
563249,SUPERMARKET_3_188,SUPERMARKET,NYC_1,201105,2.38


# Item Sales

## Pré-processamento e exploração de dados

In [ ]:
cols_df_sales = pd.DataFrame(df_sales.columns, columns=['Column Names'])

display(cols_df_sales)

,Column Names
0,id
1,item
2,category
3,department
4,store
...,...
1915,d_1909
1916,d_1910
1917,d_1911
1918,d_1912


In [ ]:
#primeiras 5 linhas do df_sales
df_sales.head()

In [ ]:
# Cinco ultimas linhas
df_sales.tail()

In [ ]:
df_sales.shape

In [ ]:
df_sales.dtypes

In [ ]:
df_sales.describe()

In [ ]:
#df_sales.info(verbose=True)

In [ ]:
#verificando valores em falta
soma_missings_por_coluna = df_sales.isnull().sum()
soma_missings_por_coluna

id            0
item          0
category      0
department    0
store         0
             ..
d_1909        0
d_1910        0
d_1911        0
d_1912        0
d_1913        0
Length: 1920, dtype: int64

### Separando o item Sales por dados Categoricos

In [ ]:
df_sales_cat = df_sales.iloc[:,0:7]
df_sales_cat

,id,item,category,department,store,store_code,region
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
...,...,...,...,...,...,...,...
1319,HOME_&_GARDEN_2_224_NYC_1,HOME_&_GARDEN_2_224,HOME_&_GARDEN,HOME_&_GARDEN_2,Greenwich_Village,NYC_1,New York
1320,HOME_&_GARDEN_2_225_NYC_1,HOME_&_GARDEN_2_225,HOME_&_GARDEN,HOME_&_GARDEN_2,Greenwich_Village,NYC_1,New York
1321,HOME_&_GARDEN_2_226_NYC_1,HOME_&_GARDEN_2_226,HOME_&_GARDEN,HOME_&_GARDEN_2,Greenwich_Village,NYC_1,New York
1322,HOME_&_GARDEN_2_227_NYC_1,HOME_&_GARDEN_2_227,HOME_&_GARDEN,HOME_&_GARDEN_2,Greenwich_Village,NYC_1,New York


### Separando o item Sales por dados Numericos

In [ ]:
#df_units_pday = df_sales.iloc[:, 7:].T.reset_index().rename(columns={"index": "d"})
#df_units_pday

### Função para observar cada coluna do df

In [ ]:
def sales_func(data, column, count=True):
    unique_values = data[column].unique()
    null_count = data[column].isnull().sum()
    value_counts = data[column].value_counts()

    print(f'Quantidade de valores únicos: {len(unique_values)}')
    print(f'\nQuais são os valores únicos: {unique_values}')
    print(f'\nQuantidade de valores nulos: {null_count}')
    print(f'\nQuantidade por opção: \n{value_counts}')

#### Coluna id

In [ ]:
sales_func(df_sales_cat, 'id')

#### Coluna item

In [ ]:
sales_func(df_sales_cat, 'item')

In [ ]:
sales_func(df_sales_cat, 'category')

In [ ]:
sales_func(df_sales_cat, 'department')

In [ ]:
sales_func(df_sales_cat, 'store')

In [ ]:
sales_func(df_sales_cat, 'store_code')

In [ ]:
sales_func(df_sales_cat, 'region')

In [ ]:
df_group = df_sales.groupby("region")["store_code"].value_counts()
print(df_group)

In [ ]:
df_sales['department'].unique()

In [ ]:
df_sales['id'].unique()

# Daily Calendar with Events

In [ ]:
df_calendar.shape

In [ ]:
df_calendar.describe()

In [ ]:
cols_df_calendar = pd.DataFrame(df_calendar.columns, columns=['Column Names'])

display(cols_df_calendar)

In [ ]:
df_calendar.head()

In [ ]:
df_calendar.isnull().sum()

In [15]:
df_prices.to_csv("df_prices_final.csv", index=False)

In [16]:
df_sales.to_csv("df_sales_final.csv", index=False)

# Juntando as Tabelas


## Transformação de df_sales:

In [ ]:

# Carregue seu dataframe com Dask
df_sales_dask = dd.read_csv("/content/drive/MyDrive/ARQUIVOS TFM/item_sales_final.csv")

# 1. Defina o tamanho do chunk
chunk_size = 500

# 2. Divida as colunas d_ em chunks
d_cols = [f'd_{i}' for i in range(1, 1914)]
d_chunks = [d_cols[i:i + chunk_size] for i in range(0, len(d_cols), chunk_size)]

# 3. Realize a operação melt em cada chunk
melted_dfs = []
for d_chunk in d_chunks:
    melted_df = df_sales_dask.melt(id_vars=['id', 'item', 'category', 'department', 'store', 'store_code', 'region'],
                                   value_vars=d_chunk,
                                   var_name='d',
                                   value_name='sold_units')
    melted_dfs.append(melted_df)

# 4. Concatene os resultados
df_sales_melted_dask = dd.concat(melted_dfs, axis=0)

# Verifique os resultados - Note que você deve usar .compute() para obter os resultados, pois Dask é "lazy" por natureza
print(df_sales_melted_dask.shape[0].compute())  # Isso retornará o número de linhas
print(df_sales_melted_dask['store_code'].value_counts().compute())  # Isso retornará a contagem de valores para 'store_code'


In [9]:
# 1. Defina o tamanho do chunk
chunk_size = 500

# 2. Divida as colunas d_ em chunks
d_cols = [f'd_{i}' for i in range(1, 1914)]
d_chunks = [d_cols[i:i + chunk_size] for i in range(0, len(d_cols), chunk_size)]

# 3. Realize a operação melt em cada chunk
melted_dfs = []
for d_chunk in d_chunks:
    melted_df = df_sales.melt(id_vars=['id', 'item', 'category', 'department', 'store', 'store_code', 'region'],
                              value_vars=d_chunk,
                              var_name='d',
                              value_name='sold_units')
    melted_dfs.append(melted_df)

# 4. Concatene os resultados
df_sales_melted = pd.concat(melted_dfs, axis=0)

# Verifique os resultados
print(df_sales_melted.shape)
print(df_sales_melted['store_code'].value_counts())


(58327370, 9)
NYC_1    5832737
NYC_2    5832737
NYC_3    5832737
NYC_4    5832737
BOS_1    5832737
BOS_2    5832737
BOS_3    5832737
PHI_1    5832737
PHI_2    5832737
PHI_3    5832737
Name: store_code, dtype: int64


## Juntar com daily_calendar_with_events:

In [10]:
# Converter a coluna 'd' em ambos os dataframes para int64
df_calendar['d'] = df_calendar['d'].str.extract('(\d+)').astype(int)

# Suponhamos que cada chunk terá 50000 linhas
chunk_size = 50000
chunks = [x for x in range(0, df_sales_melted.shape[0], chunk_size)]

# Lista para armazenar os resultados de cada chunk processado
dfs = []

for start in chunks:
    # Pega o pedaço do DataFrame
    subset = df_sales_melted.iloc[start:start+chunk_size].copy()

    # Converter a coluna 'd' em inteiro para subset
    subset['d'] = subset['d'].str.extract('(\d+)').astype(int)

    # Junção com df_calendar
    merged_subset = pd.merge(subset, df_calendar, on='d', how='left')

    # Adicionar o resultado na lista
    dfs.append(merged_subset)

# Concatenar todos os resultados
df_sales_calendar = pd.concat(dfs, ignore_index=True)


In [18]:
df_sales_calendar.to_csv("df_sales_calendar.csv", index=False)

## Extrair o 'yearweek' a partir da coluna 'date':

In [11]:
df_sales_calendar['date'] = pd.to_datetime(df_sales_calendar['date'])
df_sales_calendar['yearweek'] = df_sales_calendar['date'].dt.strftime('%Y%U').astype(int)


## Juntar com item_prices:

In [12]:
df_prices['yearweek'] = df_prices['yearweek'].astype(int)

In [13]:
client = Client()  # Inicia um cluster local

# Função para otimizar os tipos de dados do DataFrame
def optimize_dataframe(df):
    # Convertendo colunas para category
    categorical_cols = ['item', 'category', 'department', 'store', 'store_code', 'region', 'weekday', 'event']
    for col in categorical_cols:
        df[col] = df[col].astype('category')

    # Convertendo int64 para int32
    df['d'] = df['d'].astype('int32')
    df['sold_units'] = df['sold_units'].astype('int32')
    df['weekday_int'] = df['weekday_int'].astype('int32')
    df['yearweek'] = df['yearweek'].astype('int32')

    # Convertendo float64 para float32
    df['sell_price'] = df['sell_price'].astype('float32')

    return df

# Função para fazer a junção (merge) em pedaços (chunks)
def merge_in_chunks_dask(df_main, df_secondary, on_columns, n_chunks=20, how='left'):
    chunk_size = len(df_main) // n_chunks
    merged_dfs = []

    # Mesmo processo de iteração, mas com algumas modificações para Dask
    for i in tqdm(range(n_chunks)):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if i != n_chunks - 1 else None

        chunk = df_main.loc[start_idx:end_idx]
        merged_chunk = chunk.merge(df_secondary, on=on_columns, how=how, suffixes=('', '_drop'))

        # Remover colunas com sufixo "_drop"
        drop_cols = [col for col in merged_chunk.columns if col.endswith('_drop')]
        merged_chunk = merged_chunk.drop(columns=drop_cols)

        # Preenche NaNs na coluna 'event' com 'Sem evento'
        merged_chunk['event'] = merged_chunk['event'].fillna('Sem evento')

        # Preenchendo NaNs na coluna 'sell_price'
        # Usamos groupby e transform para computar a média do 'sell_price' para cada 'item'
        # e depois preenchemos os NaNs com essa média
        merged_chunk['sell_price'] = merged_chunk.groupby('item')['sell_price'].transform(lambda x: x.fillna(x.mean()))

        # Atenção: Se um item inteiro está faltando no df_prices, então a média será NaN
        # Nesse caso, você pode optar por preencher com 0 ou qualquer outro valor padrão
        merged_chunk['sell_price'] = merged_chunk['sell_price'].fillna(0)

        merged_dfs.append(merged_chunk)

    return dd.concat(merged_dfs)

# Carregando os dados usando Dask (adapte o caminho do arquivo conforme necessário)
df_sales_calendar_dask = dd.read_csv("/content/drive/MyDrive/ARQUIVOS TFM/daily_calendar_with_events.csv")
df_prices_dask = dd.read_csv("/content/drive/MyDrive/ARQUIVOS TFM/item_prices_final.csv")

on_columns = ['item', 'store_code', 'yearweek']
df_final_dask = merge_in_chunks_dask(df_sales_calendar_dask, df_prices_dask, on_columns, n_chunks=20)

# Otimizando o DataFrame final
df_final_dask = df_final_dask.map_partitions(optimize_dataframe)

# Se você eventualmente precisar do DataFrame resultante em Pandas, você pode fazer:
# df_final_pandas = df_final_dask.compute()

client.close()



INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:34835
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:38209'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42997'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:40669'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:33651'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:41709', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:41709
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:55168
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:42543', name: 2, stat

FileNotFoundError: ignored

In [32]:
df_final.head(500)

,id,item,category,department,store,store_code,region,d,sold_units,date,weekday,weekday_int,event,yearweek,sell_price
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,1,0,2011-01-29,Saturday,1,Sem evento,201104,11.036588
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,1,0,2011-01-29,Saturday,1,Sem evento,201104,5.276506
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,1,0,2011-01-29,Saturday,1,Sem evento,201104,3.943870
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,1,0,2011-01-29,Saturday,1,Sem evento,201104,5.981392
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,1,0,2011-01-29,Saturday,1,Sem evento,201104,3.833366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,ACCESORIES_2_080_NYC_1,ACCESORIES_2_080,ACCESORIES,ACCESORIES_2,Greenwich_Village,NYC_1,New York,1,0,2011-01-29,Saturday,1,Sem evento,201104,5.289121
496,ACCESORIES_2_081_NYC_1,ACCESORIES_2_081,ACCESORIES,ACCESORIES_2,Greenwich_Village,NYC_1,New York,1,0,2011-01-29,Saturday,1,Sem evento,201104,2.733332
497,ACCESORIES_2_082_NYC_1,ACCESORIES_2_082,ACCESORIES,ACCESORIES_2,Greenwich_Village,NYC_1,New York,1,0,2011-01-29,Saturday,1,Sem evento,201104,1.170000
498,ACCESORIES_2_083_NYC_1,ACCESORIES_2_083,ACCESORIES,ACCESORIES_2,Greenwich_Village,NYC_1,New York,1,0,2011-01-29,Saturday,1,Sem evento,201104,3.679843


In [33]:
df_final.shape

(58327370, 15)

In [37]:
df_final.dtypes

id                     object
item                   object
category               object
department             object
store                  object
store_code             object
region                 object
d                       int64
sold_units              int64
date           datetime64[ns]
weekday                object
weekday_int             int64
event                  object
yearweek                int64
sell_price            float64
dtype: object

In [20]:
def df_final_func(data, column, count=True):
    unique_values = data[column].unique()
    null_count = data[column].isnull().sum()
    value_counts = data[column].value_counts()

    print(f'Quantidade de valores únicos: {len(unique_values)}')
    print(f'\nQuais são os valores únicos: {unique_values}')
    print(f'\nQuantidade de valores nulos: {null_count}')
    print(f'\nQuantidade por opção: \n{value_counts}')

In [36]:
df_final_func(df_final, 'sell_price')

Quantidade de valores únicos: 4165

Quais são os valores únicos: [11.03658793  5.27650627  3.94387013 ... 15.91       18.05
 34.53      ]

Quantidade de valores nulos: 0

Quantidade por opção: 
2.38     1566303
3.58     1432370
3.00     1246481
1.20     1058320
4.78     1002713
          ...   
15.16          7
21.51          7
1.51           7
18.05          1
34.53          1
Name: sell_price, Length: 4165, dtype: int64


In [35]:
#verificando valores em falta
soma_missings_por_coluna = df_final.isnull().sum()
soma_missings_por_coluna


id             0
item           0
category       0
department     0
store          0
store_code     0
region         0
d              0
sold_units     0
date           0
weekday        0
weekday_int    0
event          0
yearweek       0
sell_price     0
dtype: int64

## Exportar df_final

In [22]:
df_final.to_csv("df_final.csv", index=False)


# Verificar quais  produtos que não são mais tão populares

In [16]:
# Transformando a coluna 'date' em datetime
df_final['date'] = pd.to_datetime(df_final['date'])

# Definindo o período de tempo para os últimos três meses e três meses antes disso
end_date = df_final['date'].max()
start_date_last_3_months = end_date - pd.DateOffset(months=3)
start_date_prev_3_months = start_date_last_3_months - pd.DateOffset(months=3)

# Filtrando o DataFrame para os produtos vendidos nos últimos 6 meses
last_6_months_sales = df_final[df_final['date'] > start_date_prev_3_months]

# Calculando a média de vendas por produto nos últimos três meses e nos três meses anteriores
avg_sales_last_3_months = last_6_months_sales[last_6_months_sales['date'] > start_date_last_3_months].groupby('item')['sold_units'].mean()
avg_sales_prev_3_months = last_6_months_sales[last_6_months_sales['date'] <= start_date_last_3_months].groupby('item')['sold_units'].mean()

# Juntando as duas séries e calculando a diferença percentual
sales_comparison = pd.concat([avg_sales_prev_3_months, avg_sales_last_3_months], axis=1, keys=['Prev_3_months', 'Last_3_months'])
sales_comparison['Sales_decline_percent'] = ((sales_comparison['Prev_3_months'] - sales_comparison['Last_3_months']) / sales_comparison['Prev_3_months']) * 100

# Filtrando produtos que tiveram uma queda de, digamos, mais de 20% nas vendas
less_popular_products = sales_comparison[sales_comparison['Sales_decline_percent'] > 20]

# Ordenando o DataFrame pelo declínio percentual das vendas de forma descendente
less_popular_products_sorted = less_popular_products.sort_values(by='Sales_decline_percent', ascending=False)

print(less_popular_products_sorted)


                     Prev_3_months  Last_3_months  Sales_decline_percent
item                                                                    
SUPERMARKET_3_441         0.114130       0.000000             100.000000
SUPERMARKET_3_444         0.208696       0.000000             100.000000
HOME_&_GARDEN_2_108       0.221739       0.001099              99.504417
ACCESORIES_1_236          0.634783       0.008791              98.615084
HOME_&_GARDEN_2_202       0.079348       0.001099              98.615084
...                            ...            ...                    ...
ACCESORIES_1_002          0.331522       0.263736              20.446766
HOME_&_GARDEN_1_194       1.377174       1.096703              20.365664
SUPERMARKET_3_505         3.017391       2.407692              20.206163
HOME_&_GARDEN_2_087       0.067391       0.053846              20.099256
HOME_&_GARDEN_1_005       1.525000       1.219780              20.014412

[290 rows x 3 columns]


## Verificar os produtos mais populares entre as cidades

In [ ]:
N = 5

# Agrupando por loja, produto e somando as vendas
store_product_sales = df_final.groupby(['store', 'item'])['sold_units'].sum().reset_index()

# Lista para armazenar os produtos mais populares por loja
top_products_list = []

for store in store_product_sales['store'].unique():
    store_sales = store_product_sales[store_product_sales['store'] == store]

    # Ordenando os produtos por vendas e pegando os top N produtos mais populares
    top_products = store_sales.sort_values(by='sold_units', ascending=False).head(N)

    top_products_list.append(top_products)

# Concatenando os resultados para criar o DataFrame final
top_products_per_store = pd.concat(top_products_list, axis=0)

print(top_products_per_store[['store', 'item', 'sold_units']])




                   store               item  sold_units
2810            Back_Bay  SUPERMARKET_3_586      150122
2314            Back_Bay  SUPERMARKET_3_090      114854
2476            Back_Bay  SUPERMARKET_3_252       86632
2779            Back_Bay  SUPERMARKET_3_555       77673
2601            Back_Bay  SUPERMARKET_3_377       73797
5363            Brooklyn  SUPERMARKET_3_090       52264
5859            Brooklyn  SUPERMARKET_3_586       32557
5860            Brooklyn  SUPERMARKET_3_587       24333
5525            Brooklyn  SUPERMARKET_3_252       23417
6078            Brooklyn  SUPERMARKET_3_808       21282
8412   Greenwich_Village  SUPERMARKET_3_090      127203
8908   Greenwich_Village  SUPERMARKET_3_586       87691
8574   Greenwich_Village  SUPERMARKET_3_252       74971
8442   Greenwich_Village  SUPERMARKET_3_120       61899
9035   Greenwich_Village  SUPERMARKET_3_714       54080
11957             Harlem  SUPERMARKET_3_586       63416
11623             Harlem  SUPERMARKET_3_252     

## Verifivar os produtos mais populares entre as lojas, levando em conta que os preços desses produtos podem ser diferentes de uma loja para outra.

In [ ]:
print("Colunas em df_final:", df_final.columns)
print("Colunas em df_prices:", df_prices.columns)


Colunas em df_final: Index(['id', 'item', 'category_x', 'department', 'store', 'store_code',
       'region', 'd', 'sold_units', 'date', 'weekday', 'weekday_int', 'event',
       'yearweek', 'category_y', 'sell_price'],
      dtype='object')
Colunas em df_prices: Index(['item', 'category', 'store_code', 'yearweek', 'sell_price'], dtype='object')


In [23]:
# Juntando o df_final com os preços
df_merged = pd.merge(df_final, df_prices, on=['store_code', 'item', 'yearweek'], how='left')

# Calculando o faturamento
df_merged['revenue'] = df_merged['sold_units'] * df_merged['sell_price']

# Agrupando por loja e produto e somando as vendas e receita
store_product_sales_revenue = df_merged.groupby(['store', 'item']).agg({'sold_units': 'sum', 'revenue': 'sum'}).reset_index()

# Identificar os produtos mais populares globalmente (em todas as lojas)
global_top_products = store_product_sales_revenue.groupby('item').agg({'sold_units': 'sum', 'revenue': 'sum'}).sort_values(by='sold_units', ascending=False).head(N)

# Para os produtos mais populares, vamos verificar os preços em diferentes lojas
popular_products = global_top_products['item'].tolist()
price_comparison = df_prices[df_prices['item'].isin(popular_products)]


KeyError: ignored

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58327370 entries, 0 to 2916377
Data columns (total 16 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           object        
 1   item         object        
 2   category_x   object        
 3   department   object        
 4   store        object        
 5   store_code   object        
 6   region       object        
 7   d            int64         
 8   sold_units   int64         
 9   date         datetime64[ns]
 10  weekday      object        
 11  weekday_int  int64         
 12  event        object        
 13  yearweek     int64         
 14  category_y   object        
 15  sell_price   float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(10)
memory usage: 7.4+ GB
